# BASIC CNN 
https://www.youtube.com/watch?v=doT7koXt9vw&list=PL5TJqBvpXQv6cd1kade4kjwr8_QQj8j3k&index=3

### IMPORTS

In [ ]:
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
from tqdm import tqdm
from torch import nn
from torchsummary import summary
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
import sklearn.metrics as metrics

### DATALOADER

In [ ]:
DATA_DIR = '/home/diogo/Documentos/IC/CNN_Learning/dataset'
CATEGORIES = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:
prep_transform = T.Compose([
                    T.ToTensor(),
                    T.Normalize(
                        (0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)
                    )
                  ])

tensor_train = CIFAR10(DATA_DIR, train=True, download=True,
                         transform=prep_transform)
tensor_test = CIFAR10(DATA_DIR, train=False, download=True,
                         transform=prep_transform)


In [ ]:
batch_size = 64

train_loader = DataLoader(tensor_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(tensor_test, batch_size=batch_size, shuffle=False)

### MODEL

In [ ]:
class CNN_net(nn.Module):
  def __init__(self):
      super().__init__()
      self.convlayers = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=(3, 3)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Conv2d(16, 32, kernel_size=(3, 3)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

      )

      self.linearlayers = nn.Sequential(
          nn.Linear(1152, 256),
          nn.ReLU(),
          nn.Linear(256, 100)
      )

  def forward(self, x):
      x = self.convlayers(x)
      x = torch.flatten(x, 1)
      return self.linearlayers(x)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Rodando na {device}")

In [ ]:
cnn_model = CNN_net().to(device)

In [ ]:
summary(cnn_model, (3, 32, 32))

In [ ]:
cnn_loss_func = nn.CrossEntropyLoss()
cnn_optimizer = torch.optim.SGD(cnn_model.parameters(), lr=0.001)

### TRAINING

In [ ]:
def plot_loss(train_losses, test_losses):
    fig = plt.figure(figsize=(13,5))
    ax = fig.gca()
    plt.ion()
    ax.plot(train_losses, label="Train loss", color = "green")
    ax.plot(test_losses, label="Test loss", color = "purple")
    ax.legend(fontsize="16")
    ax.set_xlabel("Iteration", fontsize="16")
    ax.set_ylabel("Loss", fontsize="16")
    ax.set_title("Loss vs iterations", fontsize="16");
    plt.show()

In [ ]:

epochs = 11
conv_train_losses = []
conv_test_losses = []

for t in range(epochs):
    
    train_loss = 0.0
    test_loss = 0.0

    for img, label in tqdm(train_loader, desc=f'Epoch {t}/{epochs-1}'):
        
        cnn_optimizer.zero_grad()
        
        img,label = img.to(device) ,label.to(device)
        
        pred = cnn_model(img)
        loss = cnn_loss_func(pred, label)
        loss.backward()
        cnn_optimizer.step()
        
        train_loss += loss.item()
        
    train_loss = train_loss/len(train_loader)
    conv_train_losses.append(train_loss)
    
    with torch.no_grad():
        for img, labels in test_loader:
            
            img, labels = img.to(device), labels.to(device)

            pred = cnn_model(img)
            loss = cnn_loss_func(pred, labels)
            test_loss += loss.item()
        
    test_loss = test_loss / len(test_loader)
    conv_test_losses.append(test_loss)
    
    #if t % 10 == 0:
    print(f"Epoch: {t}; Train Loss: {train_loss}")
    #    if t != 0:
    plot_loss(conv_train_losses,conv_test_losses)

### VALIDATION

In [ ]:
cnn_model.eval()
y_true = [labels for _ , labels in test_loader]
y_pred = []
with torch.no_grad():
        for img, labels in test_loader:
            
            img, labels = img.to(device), labels.to(device)

            pred = cnn_model(img)
            _, pred = torch.max(cnn_model(img), 1)
            y_pred.append(pred)
            
            
y_true = torch.cat(y_true)
y_pred = torch.cat(y_pred)
print(y_pred)

#### Accuracy

### <img src="https://hasty.ai/media/pages/docs/mp-wiki/metrics/accuracy/8da77ede45-1684142766/12.webp" alt="Formula" width=600 height=100 >

In [ ]:
accuracy = metrics.accuracy_score(y_true, y_pred)
print(accuracy)

#### Precision
### <img src="https://miro.medium.com/v2/resize:fit:888/1*C3ctNdO0mde9fa1PFsCVqA.png" alt="Formula" width=400 height=150 >

In [ ]:
precision = metrics.precision_score(y_true, y_pred , average= 'micro')
print(precision)

#### Recall

### <img src="https://miro.medium.com/v2/resize:fit:836/1*dXkDleGhA-jjZmZ1BlYKXg.png" alt="Formula" width=400 height=150 >

In [ ]:
recall = metrics.recall_score(y_true, y_pred,average= 'micro')
print(recall)

#### F1 Score

### <img src="https://miro.medium.com/v2/resize:fit:564/format:webp/1*T6kVUKxG_Z4V5Fm1UXhEIw.png" alt="Formula" width=350 height=100 >

In [ ]:
f1 = metrics.f1_score(y_true, y_pred,average= 'micro')
print(f1)

In [ ]:
report_table = {
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
}

report_table_df = pd.DataFrame(report_table)

fig, ax = plt.subplots(figsize=(20, 1))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=report_table_df.values, colLabels=report_table_df.columns, cellLoc='center', loc='center')
table.scale(1,2)

for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_text_props(weight='bold')
        
plt.show()



In [ ]:
classification_report = metrics.classification_report(y_true, y_pred,target_names=CATEGORIES)
print(classification_report)